In [23]:
from ibm_watson_machine_learning import APIClient

In [24]:
wml_credential = {
    'url': 'https://eu-de.ml.cloud.ibm.com',
    'apikey': 'kx-42wZuXhAyU21dDBA3RL9tcSwqk-GHPd3SUMEjwbeO'
}

In [25]:
client = APIClient(wml_credential)

Python 3.7 and 3.8 frameworks are deprecated and will be removed in a future release. Use Python 3.9 framework instead.


In [26]:
client

In [27]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  --------------  ------------------------
ID                                    NAME            CREATED
b9b9359c-227c-461e-a909-667a401ef747  Nalaiya thiran  2022-10-29T15:04:32.720Z
------------------------------------  --------------  ------------------------


In [28]:
space_id = "b9b9359c-227c-461e-a909-667a401ef747"
client.set.default_space(space_id)

'SUCCESS'

In [29]:
software_space_id = client.software_specifications.get_id_by_name("tensorflow_rt22.1-py3.9")
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [30]:
software_space_id

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [31]:
model_details = client.repository.store_model(
    model='model.tgz',
    meta_props={
    client.repository.ModelMetaNames.NAME:"CVD predictor",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_id }
)

In [32]:
client.repository.list()

------------------------------------  -------------  ------------------------  --------------  -----
GUID                                  NAME           CREATED                   FRAMEWORK       TYPE
78b2d80c-8691-4cdf-8161-60f437a0a8e0  CVD predictor  2022-11-02T14:43:42.002Z  tensorflow_2.7  model
cb095664-3d62-45e3-92af-72269d041e86  CVD predictor  2022-11-01T09:11:20.002Z  tensorflow_2.7  model
------------------------------------  -------------  ------------------------  --------------  -----


In [33]:
hardware_spec_id = "c076e82c-b2a7-4d20-9c0f-1f0c2fdf5a24"
model_id = "cb095664-3d62-45e3-92af-72269d041e86"
dep_details = client.deployments.create(artifact_uid=model_id,meta_props={
     client.deployments.ConfigurationMetaNames.NAME:"Nalaiya_thiran_deployment",
     client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { "id": hardware_spec_id},
     client.deployments.ConfigurationMetaNames.ONLINE:{}})

Deployment creation failed. Error: 402. {"trace":"097634f18bf5664ebaeb7c529a3498d2","errors":[{"code":"instance_quota_exceeded","message":"This deployment cannot be processed because it exceeds the allocated capacity unit hours (CUH). Increase the compute resources for this job and try again."}]}


{"trace":"097634f18bf5664ebaeb7c529a3498d2","errors":[{"code":"instance_quota_exceeded","message":"This deployment cannot be processed because it exceeds the allocated capacity unit hours (CUH). Increase the compute resources for this job and try again."}]}


--------------------------
Deployment creation failed
--------------------------




WMLClientError: Deployment creation failed. Error: 402. {"trace":"097634f18bf5664ebaeb7c529a3498d2","errors":[{"code":"instance_quota_exceeded","message":"This deployment cannot be processed because it exceeds the allocated capacity unit hours (CUH). Increase the compute resources for this job and try again."}]}

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
img = np.expand_dims(image.img_to_array(
            image.load_img("ecg1.png", target_size=(64, 64), color_mode="grayscale")), axis=0)

In [ ]:
scoring_payload = {
 client.deployments.ScoringMetaNames.INPUT_DATA: [{
        "values": img
     }]
 }

In [ ]:
dep_id = "2ef1e397-a035-4559-a99f-83a5231498b1"

In [ ]:
client.deployments.score(deployment_id=dep_id,meta_props=scoring_payload)